In [111]:
import sqlalchemy as sql
import random
import pandas as pd

data = pd.read_excel('./data.xlsx')
backup = data.copy()


In [ ]:
# Lista de cidades do RJ
cidades_rj = [
    "Rio de Janeiro", "Niterói", "Petrópolis", "Nova Friburgo", "Teresópolis",
    "Volta Redonda", "Resende", "Barra Mansa", "Macaé", "Campos dos Goytacazes",
    "Angra dos Reis", "Cabo Frio", "Araruama", "São Gonçalo", "Itaboraí",
    "Magé", "Queimados", "Belford Roxo", "Duque de Caxias", "Nova Iguaçu"
]

# Lista de cidades de SP
cidades_sp = [
    "São Paulo", "Campinas", "Santos", "São Bernardo do Campo", "São José dos Campos",
    "Ribeirão Preto", "Sorocaba", "Santo André", "Osasco", "Jundiaí",
    "Bauru", "Piracicaba", "Franca", "Guarulhos", "Diadema",
    "Barueri", "Taboão da Serra", "Mogi das Cruzes", "Taubaté", "Suzano"
]

todas_cidades = cidades_rj + cidades_sp

estados = ['São Paulo', 'Rio de Janeiro']

# inserindo todas as cidades de SP e RJ na coluna 'Cidade'
data['Cidade'] = [random.choice(todas_cidades) for _ in range(data.shape[0])]
data['Estado'] = [random.choice(estados) for _ in range(data.shape[0])]
print(data)

In [ ]:
data['Sigla do estado'] = data['Estado'].apply(lambda x: 'SP' if x == 'São Paulo' else 'RJ')

In [ ]:
data['Idade'] = [random.randint(18, 60) for _ in range(data.shape[0])]

In [ ]:
names = [
    "Ana", "João", "Maria", "Pedro", "Lucas",
    "Beatriz", "Carlos", "Júlia", "Rafael", "Camila",
    "Gabriel", "Fernanda", "Matheus", "Larissa", "Bruno",
    "Mariana", "Henrique", "Luana", "Felipe", "Letícia"
]

data['Name'] = [random.choice(names) for _ in range(data.shape[0])]

In [ ]:
# conectando em um banco de dados
engine = sql.create_engine('sqlite:///:memory:')
conn = engine.connect()
data.drop(columns=['ID.1', 'ID.2', 'ID.3', 'ID'], inplace=True)
data.to_sql('clientes', conn, index=False, if_exists='replace')

In [ ]:
# função para consultas
def query(query, connection):
    query = pd.read_sql(query, connection)
    print(query)
    return query

In [ ]:
# Consultando quantidade de nomes repetidos
q = '''
    select name, count(name) as nomes_repetidos
    from clientes
    group by name
    having count(name) > 1;
'''
query(q, engine)

In [112]:
# Criando coluna 'tipo_cliente' atraves dos valores que ele tem no Saldo na conta
q = '''
    select
        c."name",
        c."Saldo na conta",
        c."Estado",
        case
            when c."Saldo na conta" > 100000 then 'Premium'
            when c."Saldo na conta" between 50000 and 100000 then 'Regular'
            else 'Basico'
        end as "tipo_cliente"
    from clientes c
    where
        c."Estado" = 'São Paulo'
        and c."name" like 'L%';

'''
query(q, engine)

new_data = pd.read_sql(q, engine)
data['tipo_cliente'] = new_data['tipo_cliente']

       Name  Saldo na conta     Estado tipo_cliente
0   Letícia           57734  São Paulo      Regular
1     Luana           14467  São Paulo       Basico
2   Letícia           31513  São Paulo       Basico
3   Letícia            1703  São Paulo       Basico
4   Letícia           66152  São Paulo      Regular
5     Lucas           67592  São Paulo      Regular
6     Lucas           77604  São Paulo      Regular
7     Luana           22520  São Paulo       Basico
8     Luana           32678  São Paulo       Basico
9     Luana            1235  São Paulo       Basico
10    Lucas           96885  São Paulo      Regular
11    Luana           93737  São Paulo      Regular
12  Letícia           52487  São Paulo      Regular
13  Letícia           44165  São Paulo       Basico
14  Larissa           76116  São Paulo      Regular
15    Lucas           17132  São Paulo       Basico
16  Larissa           97215  São Paulo      Regular
17    Luana           15959  São Paulo       Basico
18  Letícia 

In [110]:

data.to_excel('./data.xlsx', index_label="ID")